<a href="https://colab.research.google.com/github/tylerlawer/421-HW5/blob/main/NeuralNetPredictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip3 uninstall gdown -y && pip3 install gdown
import gdown

url = "https://drive.google.com/uc?id=1UPXnC2oy20r4Yesw7lPuDRHlsoR0l0DV"
output = "data.csv"
gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1UPXnC2oy20r4Yesw7lPuDRHlsoR0l0DV
To: /content/data.csv
100%|██████████| 8.96M/8.96M [00:00<00:00, 70.4MB/s]


'data.csv'

In [2]:
## import the data
import pandas as pd

df = pd.read_csv("data.csv")

print(df)

      xRot   yRot                block_1  block_1_dist                block_2  \
0    81.45  26.98   minecraft:oak_leaves             6   minecraft:oak_planks   
1    59.47   2.02   minecraft:oak_planks             4        minecraft:water   
2   -58.81   3.46         minecraft:dirt           159        minecraft:stone   
3    61.45  36.54   minecraft:oak_leaves            40        minecraft:stone   
4   -44.43  84.42  minecraft:grass_block            56        minecraft:stone   
..     ...    ...                    ...           ...                    ...   
115  23.38  11.33   minecraft:oak_planks            11        minecraft:water   
116   1.92  50.17         minecraft:dirt            72   minecraft:oak_planks   
117 -64.33  83.93   minecraft:oak_planks            21  minecraft:grass_block   
118  -1.77   5.55        minecraft:stone           146        minecraft:stone   
119  11.65  76.97          minecraft:air           113   minecraft:oak_planks   

     block_2_dist          

In [3]:
## Getting all values of non-numeric types only
df.info()
non_numeric_values = df.select_dtypes(exclude=["float", "int"])
# print(non_numeric_values)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Columns: 7202 entries, xRot to block_3600_dist
dtypes: float64(2), int64(3600), object(3600)
memory usage: 6.6+ MB


In [4]:
## Mapping all non-numeric data values to some numeric values

# parameters:
columns_to_check_for_unique_values = 20

# internal values:
block_mapping_conversions = []

print("Event log for block conversion pushes: ")
j = 1  # iterator for determining which block column to use
while(j < columns_to_check_for_unique_values+1):
  block_column = f"block_{j}"
  block_mapping = df[block_column].factorize()
  # print(f"Conversion values for {block_column}: ")
  # print(block_mapping)
  # print()
  i = 0  # iterator for checking within a single column's conversion mapping
  while i < len(block_mapping[0]):
    mapped_index = block_mapping[0][i]
    if mapped_index == -1:
      i += 1
      continue
    new_conversion = block_mapping[1][mapped_index]
    if new_conversion not in block_mapping_conversions:
      block_mapping_conversions.append(new_conversion)
      print(f"pushing {new_conversion} to values at index {len(block_mapping_conversions)-1}")
    i += 1
  j += 1

block_mapping_conversions.append('unknown')
print(f"pushing unknown to values at index {len(block_mapping_conversions)-1}")

print()
print("Final conversion values: ")
print(block_mapping_conversions)

Event log for block conversion pushes: 
pushing minecraft:oak_leaves to values at index 0
pushing minecraft:oak_planks to values at index 1
pushing minecraft:dirt to values at index 2
pushing minecraft:grass_block to values at index 3
pushing minecraft:air to values at index 4
pushing minecraft:stone to values at index 5
pushing minecraft:water to values at index 6
pushing minecraft:oak_log to values at index 7
pushing unknown to values at index 8

Final conversion values: 
['minecraft:oak_leaves', 'minecraft:oak_planks', 'minecraft:dirt', 'minecraft:grass_block', 'minecraft:air', 'minecraft:stone', 'minecraft:water', 'minecraft:oak_log', 'unknown']


In [5]:
## Converting actual data values in the dataframe to numbers using block mapping
def convertBlock(val):
  for i in range(len(block_mapping_conversions)):
    if val == block_mapping_conversions[i]:
      return i
  return -1

block_columns = [col for col in df.columns if col.startswith("block_") and not col.endswith("_dist")]
# print(block_columns)
df_numerical = df.copy() # make a backup copy of the dataframe to reference later
for col in block_columns:
  df_numerical[col] = df_numerical[col].apply(convertBlock)

print()
print(df_numerical) # the numerized dataframe


      xRot   yRot  block_1  block_1_dist  block_2  block_2_dist  block_3  \
0    81.45  26.98        0             6        1            94        7   
1    59.47   2.02        1             4        6             9        4   
2   -58.81   3.46        2           159        5            75        4   
3    61.45  36.54        0            40        5            21        4   
4   -44.43  84.42        3            56        5            41        5   
..     ...    ...      ...           ...      ...           ...      ...   
115  23.38  11.33        1            11        6            50        4   
116   1.92  50.17        2            72        1            93        1   
117 -64.33  83.93        1            21        3           110        5   
118  -1.77   5.55        5           146        5           151        5   
119  11.65  76.97        4           113        1            18        3   

     block_3_dist  block_4  block_4_dist  ...  block_3596  block_3596_dist  \
0       

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [10]:
distance_columns = [col for col in df_numerical.columns if 'dist' in col]

target_block = 'block_5'

target_distance_column = 'block_5_dist'

for col in distance_columns:
    if col != target_distance_column:
        df_numerical[col] = (df_numerical[col] - df_numerical[target_distance_column]).abs()


In [11]:
target_block = 'block_5'

X = df_numerical.drop(columns=[target_block])
y = df_numerical[target_block]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [12]:
knn = KNeighborsClassifier(n_neighbors=24, weights='distance')
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=24, weights='distance')

In [13]:
y_pred = knn.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"KNN model accuracy: {accuracy:.2f}")

KNN model accuracy: 0.38
